### Analysinng Mass Media Bias:
- Which entities are most supportive or critical of the policies in the mass media? 
- What is the ideological slant of media outlets regarding the economic and technology policies?

In [1]:
import numpy as np
import pandas as pd
import sys
import random
import progressbar
import torch
import pickle
from mytree import *
from utils import *
from treeUtil import *
import tqdm
import argparse
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torch.nn.utils import clip_grad_norm_
import functools
from sklearn.linear_model import LogisticRegression
import gensim
from gensim.models import KeyedVectors
import string
from stanfordcorenlp import StanfordCoreNLP
import nltk.tree
from ast import literal_eval
import pptree
import logging

In [2]:
from nltk.tokenize import word_tokenize
import re
import os
from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder
from logger import Logger

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/

In [3]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
from spacy.pipeline import merge_entities
nlp = en_core_web_sm.load()
nlp.add_pipe(merge_entities)

In [4]:
import copy
from sklearn.utils import shuffle
from sklearn import metrics

In [64]:
proanti = True # True if doing pro/anti (ideology classification); False if doing nuetral/non-neutral classification (Step1: Stance Detection)
w2vec = False # False if in-domain; True if general
ner = True # if replacing named entities with their entity tags
blackout = False # if blacking out the named entities
balanced = True # if dataset has to be balanced or not
undersample = True # True if undersampling, False if oversampling
non_trainable = True # if first layer of classifier is fixed
economic = True # economic or tech

In [7]:
def replace_entity(sent):
    doc = nlp(sent)
    for ent in doc.ents:
        if ent.label_=='PERSON':
            sent = sent.replace(ent.text,ent.label_)
    return sent

def blackout_entity(sent):
    doc = nlp(sent)
    for ent in doc.ents:
        if ent.label_=='PERSON':
            sent = sent.replace(ent.text,"")
    return sent

def read_into_string(filename):
    text_file = open(filename, 'r')
    lines = text_file.read().split('\n')
    if ner:
        lines = [replace_entity(sent) for sent in lines]
    if blackout:
        lines = [blackout_entity(sent) for sent in lines]
    text_file.close()
    return lines

def process(file):
    sents = read_into_string(file)
    # restricting sentence length
    sents = [s for s in sents if len(s)>14 and len(s)<500]
    return sents

In [65]:
if economic:
    policy = 'economic'
else:
    policy = 'tech'
policy

'tech'

In [9]:
# process csv to tsv
in_file = '/Users/navreetkaur/MTP/data/new/analysis/'+policy+'_mapping.csv'
out_file = '/Users/navreetkaur/MTP/data/new/analysis/'+policy+'_mapping_tsv.csv'
with open(in_file, 'r') as fi, open(out_file, 'w') as fo:
    for line in fi:
        line = line.replace(',', '\t', 2)
        fo.write(line)

In [10]:
# converting lowercased statements to truecased
statement_file = '/Users/navreetkaur/MTP/data/new/analysis/'+policy+'_statements.txt'
df = pd.read_csv('/Users/navreetkaur/MTP/data/new/analysis/'+policy+'_mapping_tsv.csv', sep='\t', header='infer')
statements = list(df['By-Statement'])
with open(statement_file,'w') as f:
    for s in statements:
        f.write(s+'\n')
# -- run truecaser --
# 'python python PredictTruecaser.py -d distributions.obj'

In [19]:
# make new tsv with processed lines
cased_statement_file = '/Users/navreetkaur/MTP/data/new/analysis/'+policy+'_statements.txt-cased'
final_cased_csv = '/Users/navreetkaur/MTP/data/new/analysis/final_'+policy+'_statements.csv'
cased_statements = read_into_string(cased_statement_file)
num_cannot_predict = []
new_cased_statements = []
for i, s in enumerate(cased_statements):
    if len(s)>14 and len(s)<500:
        new_cased_statements.append(cased_statements[i])
    else:
        num_cannot_predict.append(i)
num_cannot_predict, len(new_cased_statements)

([65, 303, 572, 1398, 1490, 1648, 1816, 1932, 2048, 2781], 2772)

In [20]:
num_cannot_predict = num_cannot_predict[:-1]
num_cannot_predict

[65, 303, 572, 1398, 1490, 1648, 1816, 1932, 2048]

In [37]:
df = df.drop(df.index[num_cannot_predict]) # delete the statements which have length >500 or <14
cased_statements = new_cased_statements
len(cased_statements), df.shape

(2772, (2772, 3))

In [40]:
df['By-Statement'] = cased_statements
df.to_csv(final_cased_csv, sep='\t', index=False)
df['tokens'] = [word_tokenize(re.sub(r'[^\w\s]|[\d]+',' ',sent)) for sent in df['By-Statement']]
df.head()

,Entity,Source,By-Statement,tokens
0,Ajay Kumar,NIE,Alleging that Modi has not fulfilled any of th...,"[Alleging, that, Modi, has, not, fulfilled, an..."
1,Ajay Kumar,TOI,Additional district magistrate Ajaykumar said ...,"[Additional, district, magistrate, Ajaykumar, ..."
2,Ajit Pawar,TOI,Asserting that the interests of farmers will b...,"[Asserting, that, the, interests, of, farmers,..."
3,Ajit Pawar,IE,", Ajitpawar had said in an insensitive comment...","[Ajitpawar, had, said, in, an, insensitive, co..."
4,Ajit Pawar,IE,B'Amid clamour for his resignation over the co...,"[B, Amid, clamour, for, his, resignation, over..."


### Make trees

In [41]:
# java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -annotators "tokenize,ssplit,pos,lemma,parse,sentiment" -port 9000 -timeout 90000
nlp = StanfordCoreNLP('http://localhost', port=9000,timeout=90000)

In [42]:
def make_tree(text):
#     print(text)
    output = nlp.annotate(text, properties={
        'annotators': 'tokenize,ssplit,pos,depparse,parse',
        'outputFormat': 'json'
    })
    output = literal_eval(output)
    try:
        tree = str(output['sentences'][0]['parse'])
    except:
        print(output,text)
        return
    # print (tree)
    parse_string = ' '.join(str(tree).split())
    # print(parse_string)
    # print ("\n\n")
    tree = nltk.tree.Tree.fromstring(parse_string)
    tree.chomsky_normal_form()
    tree.collapse_unary(collapseRoot=True,collapsePOS=True)
    nt = convertNLTK_tree(tree)
    return nt

def printLabelTree(tree):
    def inorder(node,nnode):
        if node.isLeaf:
            newnode = pptree.Node('H',nnode)
            wnode = pptree.Node(node.word,newnode)
        elif nnode is not None:
            newnode = pptree.Node('H',nnode)
            inorder(node.left,newnode)
            inorder(node.right,newnode)
        elif node.isRoot():
            newnode = pptree.Node('H')
            inorder(node.left,newnode)
            inorder(node.right,newnode)
            return newnode
        return None
    pptree.print_tree(inorder(tree.root,None))

def create_trees_using_df(df):
    tree = []
    for tokens in list(df['tokens']):
        if len(tokens)==0:
            continue
        line = ' '.join(tokens)
        line += '\n'
        tree.append(make_tree(line))
    return tree

def printlabel(root,l):
    if root:
        l.append(root.label)
#         print(root.label)
        if root.left:
            l+=printlabel(root.left,[])
#             print(printlabel(root.left))
        if root.right:
            l+=printlabel(root.right,[])
#             print(printlabel(root.right))
    return l

In [46]:
trees = create_trees_using_df(df)
len(trees), df.shape
df['tree'] = trees
trees_file = '/Users/navreetkaur/MTP/data/new/analysis/'+policy+'_trees_df.pkl'
pickle.dump(df,open(trees_file,'wb'))

Load Model

In [66]:
dic = {0:'ProAnti', 1:'General', 2:'NER', 3:'Blackout', 4:'Balance', 5:'Undersample', 6:'Fixed'}
bool_list = [proanti, w2vec, ner, blackout, balanced, undersample, non_trainable]
corpus_path = '../data/new/'+policy
namecode = policy+'_w2v'
for index, bb in enumerate(bool_list):
    namecode += '_'
    if bb:
        namecode += '1'
    else:
        namecode += '0'
    namecode += dic[index]
namecode

'tech_w2v_1ProAnti_0General_1NER_0Blackout_1Balance_0Undersample_1Fixed'

In [67]:
CUDA=False
def Var(v):
    if CUDA: return Variable(v.cuda())
    else: return Variable(v)
widgets = [progressbar.Percentage(), ' ', progressbar.Bar(), ' ', progressbar.ETA()]

In [68]:
class RecursiveNN(nn.Module):
    def __init__(self, word_embeddings, vocab, embedSize=300, numClasses=2, beta = 0.3, use_weight = True, non_trainable = non_trainable):
        super(RecursiveNN, self).__init__()
#             if (w2vec):
#                 self.embedding = nn.Embedding(len(vocab), embedSize)
#                 self.embedding.load_state_dict({'weight': w2vec_weights})
        self.embedding = nn.Embedding.from_pretrained(word_embeddings)
        self.embedding.weight.requires_grad = True
        if non_trainable:
            self.embedding.weight.requires_grad = False
        else:
            self.embedding = nn.Embedding(len(vocab), embedSize)
        self.embedding = nn.Embedding(len(vocab), embedSize)
        self.W = nn.Linear(2*embedSize, embedSize, bias=True)
        self.nonLinear = torch.tanh
        self.projection = nn.Linear(embedSize, numClasses, bias=True)
        self.nodeProbList = []
        self.labelList = []
        self.loss = Var(torch.FloatTensor([0]))
        self.V = vocab
        self.beta = beta
        self.use_weight = use_weight
        self.total_rep = None #
        self.count_rep = 0 #
        self.numClasses = numClasses

    def traverse(self, node):
        if node.isLeaf:
            if node.getLeafWord() in self.V:  # check if right word is in vocabulary
                word = node.getLeafWord()
            else:  # otherwise use the unknown token
                word = 'UNK'
            # print(self.V[word],len(self.V),word,(torch.LongTensor([int(self.V[word])])))
            currentNode = (self.embedding(Var(torch.LongTensor([int(self.V[word])]))))
        else: currentNode = self.nonLinear(self.W(torch.cat((self.traverse(node.left),self.traverse(node.right)),1)))
        currentNode = currentNode/(torch.norm(currentNode))

        assert node.label!=None
        self.nodeProbList.append(self.projection(currentNode))
#         print(node.label)
#         print(torch.LongTensor([node.label]))
        self.labelList.append(torch.LongTensor([node.label]))
        loss_weight = 1-self.beta if node.annotated else self.beta
        self.loss += (loss_weight*F.cross_entropy(input=torch.cat([self.projection(currentNode)]),target=Var(torch.cat([torch.LongTensor([node.label])]))))

        #
        if not node.isRoot():
            if self.total_rep is None:
                self.total_rep = currentNode.data.clone()
            else:
                self.total_rep += currentNode.data.clone()
            self.count_rep += 1
        #
        return currentNode   
    
    
    
    def traverse_test(self, node):
        if node.isLeaf:
            if node.getLeafWord() in self.V:  # check if right word is in vocabulary
                word = node.getLeafWord()
            else:  # otherwise use the unknown token
                word = 'UNK'
            # print(self.V[word],len(self.V),word,(torch.LongTensor([int(self.V[word])])))
            currentNode = (self.embedding(Var(torch.LongTensor([int(self.V[word])]))))
        else: currentNode = self.nonLinear(self.W(torch.cat((self.traverse_test(node.left),self.traverse_test(node.right)),1)))
        currentNode = currentNode/(torch.norm(currentNode))

#         assert node.label!=None
        self.nodeProbList.append(self.projection(currentNode))
        loss_weight = 1-self.beta if node.annotated else self.beta

        if not node.isRoot():
            if self.total_rep is None:
                self.total_rep = currentNode.data.clone()
            else:
                self.total_rep += currentNode.data.clone()
            self.count_rep += 1
        #
        return currentNode
        

    def forward(self, x):
        self.nodeProbList = []
        self.labelList = []
        self.loss = Var(torch.FloatTensor([0]))
#         self.traverse(x)
        self.traverse_test(x)
#         self.labelList = Var(torch.cat(self.labelList))
        return torch.cat(self.nodeProbList)

    def getLoss(self, tree):
        nodes = self.forward(tree)
        predictions = nodes.max(dim=1)[1]
        loss = self.loss
        return predictions,loss
    
    
    def predict(self, trees):
        pbar = progressbar.ProgressBar(widgets=widgets, maxval=len(trees)).start()
        preds = []
        for j, tree in enumerate(trees):
            nodes = self.forward(tree.root)
            predictions = nodes.max(dim=1)[1]
#                     print((predictions.cpu().data).numpy(),(predictions.cpu().data).numpy().shape)
#                     print((self.labelList.cpu().data).numpy(), (self.labelList.cpu().data).numpy().shape)
            preds.append(predictions)
            pbar.update(j)
        pbar.finish()
        return preds
    

    def getRep(self, tree):
        self.count_rep = 0
        self.total_rep = None
        self.nodeProbList = []
#         self.labelList = []
        self.loss = Var(torch.FloatTensor([0]))

        root_rep = self.traverse(tree)

        return (torch.cat((root_rep,self.total_rep/self.count_rep),1)).data.numpy().T.flatten()


    def eval_sent_lvl(self,trees,clf):
        pbar = progressbar.ProgressBar(widgets=widgets, maxval=len(trees)).start()
        n = nAll = correctRoot = correctAll = 0.0
        X_predict = []
        Y_gold = []
        for j, tree in enumerate(trees):
            tree_rep = model.getRep(tree.root)
            X_predict.append(tree_rep)
            Y_gold.append(tree.root.label)
        acc = clf.score(np.array(X_predict),np.array(Y_gold))
        return acc

Get Labels of the statements

In [72]:
df = pickle.load(open('/Users/navreetkaur/MTP/data/new/analysis/'+policy+'_trees_df.pkl', 'rb'))
test_trees = df['tree']
model = pickle.load(open("/Users/navreetkaur/MTP/tech_classifier/models/"+namecode+'.pkl','rb'))
preds = model.predict(test_trees)
preds = [p.numpy()[-1] for p in preds]

labels = []
i=0
for p in preds:w
    if p==0:
        labels.append('PRO')
    else:
        labels.append('ANTI')
        i+=1

df['label'] = labels

100% |##########################################################| Time: 0:00:08


In [74]:
pickle.dump(df,open('/Users/navreetkaur/MTP/data/new/analysis/pred_'+policy+'.pkl','wb'))

## Analysis

In [75]:
df_econ = pickle.load(open('/Users/navreetkaur/MTP/data/new/analysis/pred_economic.pkl', 'rb'))
df_tech = pickle.load(open('/Users/navreetkaur/MTP/data/new/analysis/pred_tech.pkl', 'rb'))

In [76]:
df_econ.head()

,Entity,Source,By-Statement,tokens,tree,label
0,Ajay Kumar,NIE,Alleging that Modi has not fulfilled any of th...,"[Alleging, that, Modi, has, not, fulfilled, an...",<mytree.Tree object at 0x138a9c710>,ANTI
1,Ajay Kumar,TOI,Additional district magistrate Ajaykumar said ...,"[Additional, district, magistrate, Ajaykumar, ...",<mytree.Tree object at 0x138af7358>,PRO
2,Ajit Pawar,TOI,Asserting that the interests of farmers will b...,"[Asserting, that, the, interests, of, farmers,...",<mytree.Tree object at 0x138afd128>,PRO
3,Ajit Pawar,IE,", Ajitpawar had said in an insensitive comment...","[Ajitpawar, had, said, in, an, insensitive, co...",<mytree.Tree object at 0x138b042b0>,ANTI
4,Ajit Pawar,IE,B'Amid clamour for his resignation over the co...,"[B, Amid, clamour, for, his, resignation, over...",<mytree.Tree object at 0x138b14240>,PRO


In [77]:
df_tech.head()

,Entity,Source,By-Statement,tokens,tree,label
0,Ajay Kumar,Telegraph,Higher and technical education secretary PERSO...,"[Higher, and, technical, education, secretary,...",<mytree.Tree object at 0x138a9c748>,PRO
1,Ajay Kumar,Telegraph,Higher and technical education secretary PERSO...,"[Higher, and, technical, education, secretary,...",<mytree.Tree object at 0x13dab9d68>,PRO
2,Akhil Arora,HT,Providing state-of-the-art facilities free of ...,"[Providing, state, of, the, art, facilities, f...",<mytree.Tree object at 0x13dae9278>,PRO
3,Akhil Arora,TOI,The startups and entrepreneurs will be provide...,"[The, startups, and, entrepreneurs, will, be, ...",<mytree.Tree object at 0x13e6dc2b0>,PRO
4,Akhil Arora,TOI,Weightage will be given to teams or individual...,"[Weightage, will, be, given, to, teams, or, in...",<mytree.Tree object at 0x13e6e42b0>,PRO


## Entity Wise

In [83]:
df_econ.groupby(['Entity','label']).agg(['count'])

Source By-Statement tokens  tree
                              count        count  count count
Entity                 label                                 
Ajay Kumar             ANTI       1            1      1     1
                       PRO        1            1      1     1
Ajit Pawar             ANTI       1            1      1     1
                       PRO        2            2      2     2
Akhilesh Yadav         ANTI       6            6      6     6
                       PRO        4            4      4     4
Amarinder Singh        ANTI      23           23     23    23
                       PRO       41           41     41    41
Amit Shah              PRO        3            3      3     3
Arun Jaitley           ANTI      60           60     60    60
                       PRO      217          217    217   217
Arundhati Bhattacharya ANTI       1            1      1     1
                       PRO        3            3      3     3
Arvind Kejriwal        PRO        3            3      3     3
Arvind Panagariya      PRO        1            1      1     1
Babu                   ANTI      15           15     15    15
                       PRO       24           24     24    24
Barack Obama           ANTI       1            1      1     1
                       PRO        1            1      1     1
Centre                 ANTI       1            1      1     1
Deepak                 PRO        4            4      4     4
Deepak Kumar           PRO        2            2      2     2
Deepak Parekh          PRO        1            1      1     1
Devendra Fadnavis      ANTI       7            7      7     7
                       PRO       49           49     49    49
Edappadi K.Palaniswami ANTI       3            3      3     3
                       PRO        1            1      1     1
Gandhi                 ANTI      69           69     69    69
                       PRO       61           61     61    61
Gautam Kothari         ANTI       1            1      1     1
...                             ...          ...    ...   ...
Ram                    ANTI     143          143    143   143
                       PRO      222          222    222   222
Ram Nath Kovind        PRO        2            2      2     2
Read                   ANTI       1            1      1     1
                       PRO        1            1      1     1
Sanjeev Kumar          PRO        1            1      1     1
Sarbananda Sonowal     PRO        1            1      1     1
Satya Nadella          PRO        1            1      1     1
Shashi Tharoor         ANTI       8            8      8     8
                       PRO        2            2      2     2
Shivraj Singh Chouhan  ANTI       7            7      7     7
                       PRO       26           26     26    26
Shukla                 ANTI       3            3      3     3
                       PRO        5            5      5     5
Siddaramaiah           ANTI      68           68     68    68
                       PRO      124          124    124   124
Smriti Irani           ANTI       2            2      2     2
                       PRO        9            9      9     9
Subhash Chandra Garg   ANTI       1            1      1     1
                       PRO        3            3      3     3
Sukhbir Singh Badal    PRO        2            2      2     2
Sushil Kumar Modi      PRO        9            9      9     9
Sushma Swaraj          ANTI       1            1      1     1
                       PRO        1            1      1     1
Tim Cook               ANTI       1            1      1     1
Vasundhara Raje        ANTI       1            1      1     1
                       PRO        6            6      6     6
Vijay Rupani           PRO        4            4      4     4
Yogi Adityanath        ANTI       7            7      7     7
                       PRO       28           28     28    28

[124 rows x 4 columns]

In [102]:
# df_econ_entity = pd.DataFrame({'count': df_econ.groupby(['Entity','label']).agg(['count']).size()}).reset_index()
df_econ_entity = df_econ.groupby(['Entity','label']).count()
df_econ_entity = df_econ_entity.drop(columns=['Source','tokens','tree'])
df_econ_entity.to_csv('/Users/navreetkaur/MTP/data/new/analysis/economic_entity.csv')
df_econ_entity

By-Statement
Entity                 label              
Ajay Kumar             ANTI              1
                       PRO               1
Ajit Pawar             ANTI              1
                       PRO               2
Akhilesh Yadav         ANTI              6
                       PRO               4
Amarinder Singh        ANTI             23
                       PRO              41
Amit Shah              PRO               3
Arun Jaitley           ANTI             60
                       PRO             217
Arundhati Bhattacharya ANTI              1
                       PRO               3
Arvind Kejriwal        PRO               3
Arvind Panagariya      PRO               1
Babu                   ANTI             15
                       PRO              24
Barack Obama           ANTI              1
                       PRO               1
Centre                 ANTI              1
Deepak                 PRO               4
Deepak Kumar           PRO               2
Deepak Parekh          PRO               1
Devendra Fadnavis      ANTI              7
                       PRO              49
Edappadi K.Palaniswami ANTI              3
                       PRO               1
Gandhi                 ANTI             69
                       PRO              61
Gautam Kothari         ANTI              1
...                                    ...
Ram                    ANTI            143
                       PRO             222
Ram Nath Kovind        PRO               2
Read                   ANTI              1
                       PRO               1
Sanjeev Kumar          PRO               1
Sarbananda Sonowal     PRO               1
Satya Nadella          PRO               1
Shashi Tharoor         ANTI              8
                       PRO               2
Shivraj Singh Chouhan  ANTI              7
                       PRO              26
Shukla                 ANTI              3
                       PRO               5
Siddaramaiah           ANTI             68
                       PRO             124
Smriti Irani           ANTI              2
                       PRO               9
Subhash Chandra Garg   ANTI              1
                       PRO               3
Sukhbir Singh Badal    PRO               2
Sushil Kumar Modi      PRO               9
Sushma Swaraj          ANTI              1
                       PRO               1
Tim Cook               ANTI              1
Vasundhara Raje        ANTI              1
                       PRO               6
Vijay Rupani           PRO               4
Yogi Adityanath        ANTI              7
                       PRO              28

[124 rows x 1 columns]

In [84]:
df_tech.groupby(['Entity', 'label']).agg(['count'])

Source By-Statement tokens  tree
                               count        count  count count
Entity                  label                                 
Ajay Kumar              PRO        2            2      2     2
Akhil Arora             PRO        3            3      3     3
Akhilesh Yadav          PRO        3            3      3     3
Amarinder Singh         PRO        3            3      3     3
Anand Kishore           PRO        1            1      1     1
Ananta Narayan Jena     ANTI       1            1      1     1
                        PRO        7            7      7     7
Angela Merkel           ANTI       1            1      1     1
Anil Baijal             PRO        3            3      3     3
Anil Vij                PRO        1            1      1     1
Arun Jaitley            ANTI       6            6      6     6
                        PRO       21           21     21    21
Arundhati Bhattacharya  PRO        1            1      1     1
Ashraf                  ANTI       1            1      1     1
                        PRO        1            1      1     1
Babu                    ANTI       3            3      3     3
                        PRO       14           14     14    14
Barack Obama            ANTI       3            3      3     3
                        PRO        6            6      6     6
Chanda Kochhar          PRO        1            1      1     1
Chandra Bose            PRO        1            1      1     1
Deepak                  ANTI       3            3      3     3
                        PRO        6            6      6     6
Deepak Kumar            ANTI       1            1      1     1
                        PRO        2            2      2     2
Devendra Fadnavis       ANTI       2            2      2     2
                        PRO        1            1      1     1
Dhawan                  PRO        1            1      1     1
Dinesh Waghmare         ANTI       1            1      1     1
Gandhi                  ANTI      10           10     10    10
...                              ...          ...    ...   ...
Rajnath Singh           PRO        7            7      7     7
Ram                     ANTI      37           37     37    37
                        PRO      148          148    148   148
Ram Nath Kovind         PRO        1            1      1     1
Randeep Singh Surjewala PRO        1            1      1     1
Read                    PRO        3            3      3     3
Sarbananda Sonowal      PRO        2            2      2     2
Satya Nadella           PRO        3            3      3     3
Shashi Tharoor          ANTI       4            4      4     4
                        PRO        8            8      8     8
Shinzo Abe              PRO        1            1      1     1
Shukla                  PRO        6            6      6     6
Siddaramaiah            ANTI      13           13     13    13
                        PRO       49           49     49    49
Smriti Irani            ANTI       5            5      5     5
                        PRO       12           12     12    12
Soumini Jain            PRO        1            1      1     1
Stefan Lofven           PRO        1            1      1     1
Sushil Kumar Modi       PRO        1            1      1     1
Syed Akbaruddin         PRO        1            1      1     1
Thampi                  PRO        1            1      1     1
Tim Cook                PRO        9            9      9     9
Vasundhara Raje         ANTI       1            1      1     1
Vijay Mallya            ANTI       1            1      1     1
                        PRO        3            3      3     3
Vijay Rupani            PRO        1            1      1     1
Vikas Swarup            PRO        1            1      1     1
Virat Kohli             ANTI       1            1      1     1
                        PRO        2            2      2     2
Xi Jinping              PRO        4            4      4     

In [101]:
df_tech_entity = df_tech.groupby(['Entity','label']).count()
df_tech_entity = df_tech_entity.drop(columns=['Source','tokens','tree'])
df_tech_entity.to_csv('/Users/navreetkaur/MTP/data/new/analysis/tech_entity.csv')
df_tech_entity

By-Statement
Entity                  label              
Ajay Kumar              PRO               2
Akhil Arora             PRO               3
Akhilesh Yadav          PRO               3
Amarinder Singh         PRO               3
Anand Kishore           PRO               1
Ananta Narayan Jena     ANTI              1
                        PRO               7
Angela Merkel           ANTI              1
Anil Baijal             PRO               3
Anil Vij                PRO               1
Arun Jaitley            ANTI              6
                        PRO              21
Arundhati Bhattacharya  PRO               1
Ashraf                  ANTI              1
                        PRO               1
Babu                    ANTI              3
                        PRO              14
Barack Obama            ANTI              3
                        PRO               6
Chanda Kochhar          PRO               1
Chandra Bose            PRO               1
Deepak                  ANTI              3
                        PRO               6
Deepak Kumar            ANTI              1
                        PRO               2
Devendra Fadnavis       ANTI              2
                        PRO               1
Dhawan                  PRO               1
Dinesh Waghmare         ANTI              1
Gandhi                  ANTI             10
...                                     ...
Rajnath Singh           PRO               7
Ram                     ANTI             37
                        PRO             148
Ram Nath Kovind         PRO               1
Randeep Singh Surjewala PRO               1
Read                    PRO               3
Sarbananda Sonowal      PRO               2
Satya Nadella           PRO               3
Shashi Tharoor          ANTI              4
                        PRO               8
Shinzo Abe              PRO               1
Shukla                  PRO               6
Siddaramaiah            ANTI             13
                        PRO              49
Smriti Irani            ANTI              5
                        PRO              12
Soumini Jain            PRO               1
Stefan Lofven           PRO               1
Sushil Kumar Modi       PRO               1
Syed Akbaruddin         PRO               1
Thampi                  PRO               1
Tim Cook                PRO               9
Vasundhara Raje         ANTI              1
Vijay Mallya            ANTI              1
                        PRO               3
Vijay Rupani            PRO               1
Vikas Swarup            PRO               1
Virat Kohli             ANTI              1
                        PRO               2
Xi Jinping              PRO               4

[117 rows x 1 columns]

## Media Source wise

In [85]:
df_econ.groupby(['Source','label']).agg(['count'])

Entity By-Statement tokens  tree
                 count        count  count count
Source    label                                 
DecH      ANTI     217          217    217   217
          PRO      586          586    586   586
HT        ANTI      66           66     66    66
          PRO      149          149    149   149
IE        ANTI     166          166    166   166
          PRO      492          492    492   492
NIE       ANTI     198          198    198   198
          PRO      483          483    483   483
TOI       ANTI      75           75     75    75
          PRO      148          148    148   148
Telegraph ANTI      55           55     55    55
          PRO      137          137    137   137

In [100]:
df_econ_media = df_econ.groupby(['Source','label']).count()
df_econ_media = df_econ_media.drop(columns=['Entity','tokens','tree'])
df_econ_media.to_csv('/Users/navreetkaur/MTP/data/new/analysis/economic_media.csv')
df_econ_media

By-Statement
Source    label              
DecH      ANTI            217
          PRO             586
HT        ANTI             66
          PRO             149
IE        ANTI            166
          PRO             492
NIE       ANTI            198
          PRO             483
TOI       ANTI             75
          PRO             148
Telegraph ANTI             55
          PRO             137

In [86]:
df_tech.groupby(['Source','label']).agg(['count'])

Entity By-Statement tokens  tree
                 count        count  count count
Source    label                                 
DecH      ANTI      50           50     50    50
          PRO      286          286    286   286
HT        ANTI      11           11     11    11
          PRO       60           60     60    60
IE        ANTI      34           34     34    34
          PRO      186          186    186   186
NIE       ANTI      39           39     39    39
          PRO      156          156    156   156
TOI       ANTI      19           19     19    19
          PRO      120          120    120   120
Telegraph ANTI      19           19     19    19
          PRO       86           86     86    86

In [97]:
df_tech_media = df_tech.groupby(['Source','label']).count()
df_tech_media = df_tech_media.drop(columns=['Entity','tokens','tree'])
df_tech_media.to_csv('/Users/navreetkaur/MTP/data/new/analysis/tech_media.csv')

In [98]:
df_tech_media

By-Statement
Source    label              
DecH      ANTI             50
          PRO             286
HT        ANTI             11
          PRO              60
IE        ANTI             34
          PRO             186
NIE       ANTI             39
          PRO             156
TOI       ANTI             19
          PRO             120
Telegraph ANTI             19
          PRO              86